In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master('local[*]') \
        .appName('Testing-Spark') \
        .getOrCreate()

22/02/22 15:21:43 WARN Utils: Your hostname, sebastian2296 resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlp2s0)
22/02/22 15:21:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/22 15:21:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/22 15:21:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

In [4]:
df = spark.read.option('header', 'true').csv('fhvhv_tripdata_2021-01.csv')

In [5]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

In [6]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [7]:
import pandas as pd

df_pandas = pd.read_csv('head.csv')
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [8]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [9]:
from pyspark.sql import types

In [10]:
schema = types.StructType([

    types.StructField('hvfhs_license_num',types.StringType(),True),
    types.StructField('dispatching_base_num',types.StringType(),True),
    types.StructField('pickup_datetime',types.TimestampType(),True),
    types.StructField('dropoff_datetime',types.TimestampType(),True),
    types.StructField('PULocationID',types.IntegerType(),True),
    types.StructField('DOLocationID',types.IntegerType(),True),
    types.StructField('SR_Flag',types.StringType(),True)
]
)

In [11]:
df = spark.read.option('header', 'true').schema(schema).csv('fhvhv_tripdata_2021-01.csv')

In [12]:
df.head(2)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None)]

In [13]:
df = df.repartition(24)

In [14]:
#df.write.parquet('fhvhv/2021/01/')

## Creating a Dataframe with Parquet files 

In [15]:
df_from_parquet = spark.read.parquet('fhvhv/2021/01/')

In [16]:
df_from_parquet.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [17]:
# Transformations (lazy)
x = df_from_parquet.select(['pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID']) \
    .filter(df_from_parquet['hvfhs_license_num'] == 'HV0003')

In [18]:
# Actions (eagerb)
x.show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-02 20:47:47|2021-01-02 21:13:43|         250|         258|
|2021-01-04 19:40:17|2021-01-04 19:57:29|         112|          68|
|2021-01-03 21:33:00|2021-01-03 22:01:58|         225|         170|
|2021-01-02 20:20:00|2021-01-02 20:28:10|         149|         165|
|2021-01-01 03:17:59|2021-01-01 03:48:44|          74|          39|
|2021-01-03 21:54:18|2021-01-03 22:07:02|         213|          60|
|2021-01-05 12:34:34|2021-01-05 12:57:34|          79|         138|
|2021-01-01 18:03:05|2021-01-01 18:20:14|          69|         168|
|2021-01-02 21:36:19|2021-01-02 21:45:19|         136|         119|
|2021-01-05 07:50:26|2021-01-05 08:10:07|          33|         138|
|2021-01-01 22:20:16|2021-01-01 22:35:39|          82|         135|
|2021-01-05 10:07:30|2021-01-05 10:32:20|       

### Default Available Functions in Spark

In [19]:
from pyspark.sql import functions as F

In [20]:
df_from_parquet.withColumn('pickup_date', F.to_date(df_from_parquet['pickup_datetime'])) \
                .withColumn('dropoff_date', F.to_date(df_from_parquet['dropoff_datetime'])) \
                .select(['pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID']) \
                .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-02|  2021-01-02|         250|         258|
| 2021-01-04|  2021-01-04|         112|          68|
| 2021-01-03|  2021-01-03|         225|         170|
| 2021-01-02|  2021-01-02|         149|         165|
| 2021-01-01|  2021-01-01|          74|          39|
| 2021-01-03|  2021-01-03|         138|         181|
| 2021-01-03|  2021-01-03|         213|          60|
| 2021-01-05|  2021-01-05|          79|         138|
| 2021-01-01|  2021-01-01|          69|         168|
| 2021-01-02|  2021-01-02|         136|         119|
| 2021-01-05|  2021-01-05|          33|         138|
| 2021-01-01|  2021-01-01|          82|         135|
| 2021-01-05|  2021-01-05|          91|          61|
| 2021-01-02|  2021-01-02|         100|         233|
| 2021-01-02|  2021-01-02|          63|          37|
| 2021-01-02|  2021-01-02|          68|       

### User defined functions

In [21]:
def crazy_stuff(base_num):
    
    try:
        num = int(base_num[1:])
    
    except ValueError:
        num = int(base_num[2:])
    
    if num % 7 == 0:
        return f's/{num:03x}'
        
    else:
        return f'e/{num:03x}'
        

In [22]:
crazy_stuff('HV0003')

'e/003'

In [23]:
from pyspark.sql import types

In [24]:
# defining udf to apply over spark dataframes

crazy_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [25]:
df_from_parquet.withColumn('pickup_date', F.to_date(df_from_parquet['pickup_datetime'])) \
                .withColumn('dropoff_date', F.to_date(df_from_parquet['dropoff_datetime'])) \
                .withColumn('base_id', crazy_udf(df_from_parquet['dispatching_base_num'])) \
                .select(['base_id','pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID']) \
                .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/b40| 2021-01-02|  2021-01-02|         250|         258|
|  s/b13| 2021-01-04|  2021-01-04|         112|          68|
|  e/acc| 2021-01-03|  2021-01-03|         225|         170|
|  e/b37| 2021-01-02|  2021-01-02|         149|         165|
|  e/b38| 2021-01-01|  2021-01-01|          74|          39|
|  e/9ce| 2021-01-03|  2021-01-03|         138|         181|
|  s/b44| 2021-01-03|  2021-01-03|         213|          60|
|  e/b3e| 2021-01-05|  2021-01-05|          79|         138|
|  e/acc| 2021-01-01|  2021-01-01|          69|         168|
|  e/b37| 2021-01-02|  2021-01-02|         136|         119|
|  e/a39| 2021-01-05|  2021-01-05|          33|         138|
|  e/b38| 2021-01-01|  2021-01-01|          82|         135|
|  e/b37| 2021-01-05|  2021-01-05|          91|          61|
|  e/b3c| 2021-01-02|  2

In [26]:
spark.version

'3.2.0'